<a href="https://colab.research.google.com/github/bhavi-23/tutorial/blob/master/ambulance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import networkx as nx
import time

# Generate synthetic traffic data
data_size = 5000
traffic_data = pd.DataFrame({
    'time_of_day': np.random.randint(0, 24, size=data_size),
    'location': np.random.randint(0, 50, size=data_size),
    'traffic_speed': np.random.randint(20, 80, size=data_size)
})

# Features and target
X = traffic_data[['time_of_day', 'location']]
y = traffic_data['traffic_speed']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#  XGBoost model
model = XGBRegressor()

#  hyperparameter grid for GridSearch
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_search.fit(X_train, y_train)

# best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Predict traffic speed for test data using the best model
predictions = best_model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, predictions)
print(f'Model MSE: {mse}')
print(f'Best Hyperparameters: {best_params}')

# graph for intersections and roads
G = nx.Graph()

# Add nodes (represent intersections)
for i in range(20):
    G.add_node(i)

# Add edges (represent roads between intersections) with random initial weights (traffic speed)
edges = [(i, i + 1, np.random.randint(20, 80)) for i in range(19)]  # Creating a linear connection for simplicity
edges += [(1, 3, np.random.randint(20, 80)), (2, 4, np.random.randint(20, 80)), (4, 5, np.random.randint(20, 80))]
G.add_weighted_edges_from(edges)

# Function to update graph with new traffic data
def update_traffic_conditions(graph, traffic_model):
    for edge in graph.edges:
        location = edge[0]
        predicted_speed = traffic_model.predict([[np.random.randint(0, 24), location]])[0]
        graph[edge[0]][edge[1]]['weight'] = predicted_speed


update_traffic_conditions(G, best_model)

# heuristic function for A* algorithm
def heuristic(a, b):
    # Simple heuristic: return the straight-line distance
    return abs(a - b)

# optimal route to the hospital using A*
def find_optimal_route(graph, start, end):
    shortest_path = nx.astar_path(graph, source=start, target=end, heuristic=heuristic, weight='weight')
    return shortest_path

# accident and hospital dataset
accident_data = {
    'Accident_ID': [1, 2, 3, 4],
    'Intersection': ['Intersection_0', 'Intersection_1', 'Intersection_2', 'Intersection_3'],
    'Hospital': ['Hospital_A', 'Hospital_B', 'Hospital_C', 'Hospital_D'],
    'Traffic_Condition': ['Moderate', 'Heavy', 'Light', 'Moderate'],
    'Estimated_Travel_Time': ['25 min', '40 min', '15 min', '20 min']
}


df = pd.DataFrame(accident_data)

# convert estimated travel time to integer
def convert_travel_time(time_str):
    return int(time_str.split()[0])

# convert to the Estimated_Travel_Time column
df['Travel_Time'] = df['Estimated_Travel_Time'].apply(convert_travel_time)

# hospital with the minimum travel time
best_hospital = df.loc[df['Travel_Time'].idxmin()]

# best hospital details
print(f"Best Hospital to reach: {best_hospital['Hospital']}")
print(f"Estimated Travel Time: {best_hospital['Estimated_Travel_Time']}")


optimal_route = find_optimal_route(G, start=0, end=19)
print(f"Optimized Route: {optimal_route}")


def display_ambulance_symbol(traffic_speed, intersection):

    if traffic_speed < 40:
        print(f"Heavy traffic detected at intersection {intersection}, displaying ambulance symbol early.")
    else:
        print(f"Light traffic at intersection {intersection}, displaying ambulance symbol closer to the ambulance.")
    print("Vehicles moving to the right lane, clearing the left lane for ambulance.")

# traffic light control and symbol display
def control_traffic_lights(route, graph):
    for intersection in route:
        if route.index(intersection) + 1 < len(route):
            next_intersection = route[route.index(intersection) + 1]
            traffic_speed = graph[intersection][next_intersection]['weight']

            # Display ambulance symbol
            display_ambulance_symbol(traffic_speed, intersection)

            # delay for vehicles to move
            time.sleep(1)

# traffic light control for optimized route
control_traffic_lights(optimal_route, G)

# Notify ambulance dispatch
def notify_hospital(hospital):
    print(f"Notification sent to {hospital}. Ambulance dispatched.")

nearest_hospital = best_hospital['Hospital']
notify_hospital(nearest_hospital)


Model MSE: 295.7305817567714
Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Best Hospital to reach: Hospital_C
Estimated Travel Time: 15 min
Optimized Route: [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Light traffic at intersection 0, displaying ambulance symbol closer to the ambulance.
Vehicles moving to the right lane, clearing the left lane for ambulance.
Light traffic at intersection 1, displaying ambulance symbol closer to the ambulance.
Vehicles moving to the right lane, clearing the left lane for ambulance.
Light traffic at intersection 3, displaying ambulance symbol closer to the ambulance.
Vehicles moving to the right lane, clearing the left lane for ambulance.
Light traffic at intersection 4, displaying ambulance symbol closer to the ambulance.
Vehicles moving to the right lane, clearing the left lane for ambulance.
Light traffic at intersection 5, displaying ambulance symbol closer to the ambulance.
Vehicles moving 